In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os

from PIL import Image
import io
import base64
import math

import random
from sklearn.model_selection import train_test_split
from shapely.geometry import Polygon

In [ ]:
def order_points_convex(points):
    """
    Orders the points to form a convex quadrilateral (rectangle).
    """
    if len(points) != 4:
        return points  # Only works for quadrilaterals
    
    # Calculate the centroid of the points
    centroid_x = sum(p[0] for p in points) / 4
    centroid_y = sum(p[1] for p in points) / 4
    
    # Sort points by angle relative to the centroid
    def angle_from_centroid(point):
        return math.atan2(point[1] - centroid_y, point[0] - centroid_x)
    
    sorted_points = sorted(points, key=angle_from_centroid)
    return sorted_points
    
def filter_images_with_4_plus_keypoints(directory):
    """
    Filters images that have polygons with more than 4 keypoints.
    Returns a list of file paths for such images.
    """
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")]
    filtered_files = []

    for filename in json_files:
        json_path = os.path.join(directory, filename)
        with open(json_path, 'r') as f:
            keypoints = json.load(f)
        
        if "shapes" not in keypoints:
            continue
        
        for shape in keypoints["shapes"]:
            if len(shape["points"]) > 4:
                filtered_files.append(json_path)
                break  # Stop checking other shapes in the same file
    
    return filtered_files
    
def is_convex(points):
    """
    Checks if the given points form a convex polygon.
    """
    if len(points) < 3:
        return False  # A polygon must have at least 3 points
    
    polygon = Polygon(points)
    return polygon.is_valid and polygon.convex_hull.equals(polygon)
    
def ensure_rectangle_shape(points):
    """
    Ensures the points form a rectangle-shaped polygon.
    """
    if len(points) != 4:
        return points  # Only works for quadrilaterals
    
    # Order the points to form a convex quadrilateral
    ordered_points = order_points_convex(points)
    
    # Check if the ordered points form a convex polygon
    if is_convex(ordered_points):
        return ordered_points
    else:
        # If not convex, adjust the points slightly
        adjusted_points = [(x + 0.01 * i, y + 0.01 * i) for i, (x, y) in enumerate(ordered_points)]
        return adjusted_points


def find_closest_points(points):
    min_distance = float('inf')
    closest_pair = None
    
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            x1, y1 = points[i]
            x2, y2 = points[j]
            distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            if distance < min_distance:
                min_distance = distance
                closest_pair = (i, j)
    
    return closest_pair
    
def replace_closest_points(points):
    if len(points) > 4:
        # Find the two closest points
        i, j = find_closest_points(points)
        # Calculate the midpoint of the two closest points
        midpoint = (
            (points[i][0] + points[j][0]) / 2,
            (points[i][1] + points[j][1]) / 2
        )
        # Replace the two closest points with the midpoint
        new_points = [point for idx, point in enumerate(points) if idx not in (i, j)]
        new_points.append(midpoint)
        return new_points
    return points
def reorder_points(points):
    """
    Reorders the points to ensure:
    - x1: top-left
    - x2: top-right
    - x3: bottom-right
    - x4: bottom-left
    """
    if len(points) != 4:
        return points  # Only works for quadrilaterals
    
    # Sort points by y-coordinate (top to bottom)
    sorted_by_y = sorted(points, key=lambda p: p[1])
    
    # Separate top and bottom points
    top_points = sorted_by_y[:2]  # Top two points (smallest y)
    bottom_points = sorted_by_y[2:]  # Bottom two points (largest y)
    
    # Sort top points by x-coordinate (left to right)
    top_points_sorted = sorted(top_points, key=lambda p: p[0])
    x1 = top_points_sorted[0]  # Top-left
    x2 = top_points_sorted[1]  # Top-right
    
    # Sort bottom points by x-coordinate (left to right)
    bottom_points_sorted = sorted(bottom_points, key=lambda p: p[0])
    x4 = bottom_points_sorted[0]  # Bottom-left
    x3 = bottom_points_sorted[1]  # Bottom-right
    
    # Return reordered points
    return [x1, x2, x3, x4]



In [ ]:
label_data = []
label_dir = '/kaggle/input/surround-view-mmu-parking-slots-dataset/main_data/labels'
label_files = os.listdir('/kaggle/input/surround-view-mmu-parking-slots-dataset/main_data/labels')

In [ ]:
for file_name in label_files:
    if file_name.endswith('.json'):
        with open(os.path.join(label_dir, file_name), 'r') as f:
            label_data.append(json.load(f))

In [ ]:
rows = []
for labels in label_data:
    for shape in labels['shapes']:
        true_flags = [key for key, value in labels['flags'].items() if value]
        if len(true_flags) > 1:
            print("more than one")
        for flag in true_flags:
            row= ({
                'flag': flag, 
                'imagePath': labels['imagePath'],
                'imageHeight': labels['imageHeight'],
                'imageWidth': labels['imageWidth'],
                'label': shape['label'],
            })
            points = shape["points"]
            if len(points) > 4:
                continue

            for i, (x, y) in enumerate(points):
                row[f'x{i+1}'] = x
                row[f'y{i+1}'] = y
            rows.append(row)

data = pd.DataFrame(rows)
data.dropna()

In [ ]:
label_map = {"occupied": 0, "vacant": 1, "unavailable": 2}
reverse = {0: "occupied", 1: "vacant", 2: "unavailable"}


In [ ]:
data.fillna(0, inplace=True)

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=28)

In [ ]:
train_data.info()

In [ ]:
# Create directories for train and val
os.makedirs("dataset/train/labels", exist_ok=True)
os.makedirs("dataset/val/labels", exist_ok=True)

In [ ]:
# Number of unique groups
num_groups = len(train_data.groupby('imagePath'))
print("Number of unique groups:", num_groups)

# Find the longest group
group_sizes = train_data.groupby('imagePath').size()
longest_group = group_sizes.idxmax()  # Group with the maximum size
longest_group_size = group_sizes.max()  # Size of the longest group

print(f"Longest group: {longest_group} with {longest_group_size} entries")

In [ ]:
def save_yolo_format(data, output_dir):
    i = 0
    for image_path, group in data.groupby('imagePath'):
        # Construct the file name for the annotation file
        label_path = os.path.join(output_dir, image_path.replace(".jpg", ".txt"))
        with open(label_path, 'w+') as f:
            for _, row in group.iterrows():
                # Image dimensions
                img_width = row["imageWidth"]
                img_height = row["imageHeight"]
        
                # Normalize coordinates
                x1 = max(min(row["x1"] / img_width, 1), 0)
                y1 = max(min(row["y1"] / img_height, 1), 0)
                x2 = max(min(row["x2"] / img_width, 1), 0)
                y2 = max(min(row["y2"] / img_height, 1), 0)
                x3 = max(min(row["x3"] / img_width, 1), 0)
                y3 = max(min(row["y3"] / img_height, 1), 0)
                x4 = max(min(row["x4"] / img_width, 1), 0)
                y4 = max(min(row["y4"] / img_height, 1), 0)
        
                # Calculate bounding box
                x_center = (x1 + x2 + x3 + x4) / 4
                y_center = (y1 + y2 + y3 + y4) / 4
                width = max(x1, x2, x3, x4) - min(x1, x2, x3, x4)
                height = max(y1, y2, y3, y4) - min(y1, y2, y3, y4)
        
                # Get class ID
                class_id = label_map[row["label"]]
        
                # YOLO format
                yolo_row = [
                    class_id, x_center, y_center, width, height,
                    x1, y1, 2, x2, y2, 2, x3, y3, 2, x4, y4, 2
                ]

                norm = [
                     x_center, y_center, width, height,
                    x1, y1,  x2, y2, x3, y3, x4, y4
                ]
                
                # Applying checks to see if data is good to go
                if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                    print(f"Invalid bounding box in {image_path}")
                    continue
                
                if width < 0 or height < 0:
                    print(f"Negative dimensions detected in {image_path}")
                    continue
                
                if any(map(lambda c: c != c or c == float('inf') or c == float('-inf') or (0 > c or c > 1), norm)):  # Check NaN or inf
                    print(f"Invalid coordinates detected in {image_path}: {norm}")
                    print(row['x4'] , img_height)
                    print(x1,x2,x3,x4,y1,y2,y3,y4)
                    continue
                    
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1):
                    print(f"Center out of bounds in {image_path}: x_center={x_center}, y_center={y_center}")
                    continue
                    
                if width <= 0 or height <= 0:
                    print(f"Invalid bounding box dimensions in {image_path}: width={width}, height={height}")
                    continue
            
                yolo_data = " ".join(map(str, yolo_row))
                f.write(yolo_data+'\n')
                f.flush()
    return yolo_data, row.imagePath

In [ ]:
def show_example_from_file(filename):
    with open(filename, 'r') as f:
        keypoints = json.load(f)
    detections = keypoints["shapes"]
    image = Image.open(filename.replace("json", "jpg").replace("labels", "images"))
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    ax.imshow(image)
    for points in detections:
        x_coords, y_coords = zip(*points["points"])
        ax.plot(x_coords + (x_coords[0],), y_coords + (y_coords[0],), marker='o', linestyle='-', label=points["label"])
        ax.fill(x_coords, y_coords, alpha=0.3)  # Optional: fill the polygon
        centroid_x = sum(x_coords) / len(x_coords)
        centroid_y = sum(y_coords) / len(y_coords)
        ax.text(centroid_x, centroid_y, points["label"], fontsize=12, color='white', ha='center', va='center')
        ax.legend()
    plt.show()

In [ ]:
TEST_FILE = "/kaggle/input/surround-view-mmu-parking-slots-dataset/main_data/labels/0926_2284.json"
show_example_from_file(TEST_FILE)

In [ ]:
# Process and save train and validation data
ex_ann_train, ex_image_train = save_yolo_format(train_data, "dataset/train/labels")
ex_ann_val, ex_image_val= save_yolo_format(val_data, "dataset/val/labels")

In [ ]:
ex_ann_train, ex_image_train

In [ ]:
# Save images to respective directories (assuming they're all in one directory)
os.makedirs("dataset/train/images", exist_ok=True)
os.makedirs("dataset/val/images", exist_ok=True)
train_images = train_data.groupby('imagePath')
val_images = val_data.groupby('imagePath')

In [ ]:
for image in train_images.groups.keys():
    os.system(f"cp {os.path.join('/kaggle/input/surround-view-mmu-parking-slots-dataset/main_data/images', image)} dataset/train/images/")  # Copy images to train folder

In [ ]:
print("Train Images Successfully Pasted")

In [ ]:
for image in val_images.groups.keys():
    os.system(f"cp {os.path.join('/kaggle/input/surround-view-mmu-parking-slots-dataset/main_data/images', image)} dataset/val/images/")  # Copy images to train folder

In [ ]:
print("Validation Images Successfully Pasted")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# Define dataset paths
train_images_path = "/kaggle/working/dataset/train/images"
train_labels_path = "/kaggle/working/dataset/train/labels"
val_images_path = "/kaggle/working/dataset/val/images"
val_labels_path = "/kaggle/working/dataset/val/labels"

# Function to display random file and corresponding label
def get_random_file_and_label(images_path, labels_path):
    random_image_file = random.choice(os.listdir(images_path))
    image_file_path = os.path.join(images_path, random_image_file)
    label_file_path = os.path.join(labels_path, random_image_file.replace(".jpg", ".txt").replace(".png", ".txt"))

    with open(label_file_path, 'r') as label_file:
        label_content = label_file.read()

    return os.path.join(images_path, random_image_file), label_content


def draw_bbox_multiple(images_annotations):
    """
    Draws bounding boxes and vertices on multiple images and displays them in subplots.
    :param images_annotations: List of tuples [(image_path, annotation), ...].
    """
    num_images = len(images_annotations)
    cols = 3  # Define the number of columns for subplots
    rows = (num_images + cols - 1) // cols  # Calculate the number of rows

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten() if num_images > 1 else [axes]

    for idx, (image_path, annotation) in enumerate(images_annotations):
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            axes[idx].axis('off')
            continue

        img = cv2.imread(image_path)
        h, w = img.shape[:2]

        data = list(map(float, annotation.split()))
        class_id = int(data[0])
        x_center, y_center, bbox_width, bbox_height = data[1:5]
        coords = [
            (data[i] * (w if i % 2 == 0 else h), data[i + 1] * (h if i % 2 == 0 else w), int(data[i + 2]))
            for i in range(5, 15, 3)
        ]

        # Filter visible points
        visible_points = [(int(x), int(y)) for x, y, visible in coords if visible == 2]

        # Draw polygon and visible points
        if len(visible_points) >= 3:
            cv2.polylines(img, [np.array(visible_points, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)
        for x, y in visible_points:
            cv2.circle(img, (x, y), radius=5, color=(0, 255, 0), thickness=-1)

        # Plot the image
        axes[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f"Class ID: {reverse[class_id]}, Image: {image_path.split('/')[-1]}")
        axes[idx].axis('off')

    # Hide any unused subplot axes
    for idx in range(num_images, len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
draw_bbox_multiple([get_random_file_and_label(train_images_path, train_labels_path) for i in range(15)])

In [ ]:
!pip install ultralytics --upgrade -q

import ultralytics
ultralytics.checks()

In [ ]:
import yaml

# Define the data structure
data = {
    "kpt_shape": [4, 3],      # list of integers
    "flip_idx": [1, 0, 3, 2], # list of integers
    "names": {0: "occupied", 1: 'vacant', 2: 'unavailable'},
    "nc": 3,
    "train": "/kaggle/working/dataset/train/images",  # train images (relative to 'path')
    "val": "/kaggle/working/dataset/val/images",      # val images (relative to 'path')
}

# Write to a YAML file
with open("dataset/data.yaml", "w") as file:
    yaml.dump(data, file, default_flow_style=False, sort_keys=False)

print("YAML file 'config.yaml' created successfully!")


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8l-pose.pt')

results = model.train(data=f"/kaggle/working/dataset/data.yaml", epochs=50, imgsz=640)

In [ ]:
cat /usr/local/lib/python3.10/dist-packages/ultralytics/utils/callbacks/raytune.py

In [ ]:
import ray
from ray import tune
from ray.air import session

if ray.train._internal.session._get_session() is not None:
    metrics = trainer.metrics
    metrics['epoch'] = trainer.epoch
    session.report(metrics)